In [9]:
from braket.circuits import Circuit, Gate, Instruction, circuit, Observable
import numpy as np
from braket.aws import AwsDevice
from colorama import Fore, Style
import ___experiments_labels___ as experimental_labels
from datetime import datetime
import time, os, pickle, traceback
from typing import Optional

# Define helper functions

In [10]:
def cool_print(a, b='', color=Fore.CYAN):
    if color.__str__().upper() == 'RED':
        color = Fore.RED
    if color.__str__().upper() == 'GREEN':
        color = Fore.GREEN
    if color.__str__().upper() == 'BLUE':
        color = Fore.BLUE

    # a is printed with color
    # b is printed without color
    if b == '':
        print(color + Style.BRIGHT + str(a) + Style.RESET_ALL)
    else:
        print(color + Style.BRIGHT + str(a) + Style.RESET_ALL, repr(b))


def get_time_now() -> str:
    """
    :return: nicely formatted string with current time
    """

    ct0 = str(datetime.today())
    ct1 = str.replace(ct0, ':', '_')
    ct2 = str.replace(ct1, '.', '_')
    ct3 = ct2[0:19]

    return ct3

def get_module_directory():
    import ___name_holder___
    name_holder = ___name_holder___.__file__
    # print(name_holder[0:-21])

    return name_holder[0:-len("___name_holder___.py")]

def get_today_date():
    return "{:%Y-%m-%d}".format(datetime.now())



def save_results(dictionary_to_save:dict,
                 directory:str,
                 saving_type:Optional[str]='pickle',
                 custom_name: Optional[str] = 'no',
                 get_cwd:Optional[bool]=False):
    if (directory != None):
        fp0 = [s for s in directory]
        if (fp0[len(fp0) - 1] != '/'):
            fp0.append('/')
        fp = ''.join(fp0)
    else:
        fp = ''
    # Time& date
    if (get_cwd):
        cwd = os.getcwd()
    else:
        cwd = ''
    ct3 = get_time_now()

    main_directory = cwd + fp

    # permission_mode=int('0777',8)
    # os.chmod(main_directory,permission_mode)
    check_dir = os.path.exists(main_directory)

    if (check_dir == False):
        try:
            os.makedirs(main_directory)
        except(FileExistsError):
            import shutil
            try:
                shutil.rmtree(main_directory)
                os.makedirs(main_directory)
            except(FileExistsError):
                os.makedirs(main_directory)
        print(
            Fore.CYAN + Style.BRIGHT + 'Attention: ' + Style.RESET_ALL + 'Directory ' + '"' + main_directory + '"' + ' was created.')

    if (custom_name == 'no'):
        file_path = str(main_directory + 'results' + '___' + str(ct3))
    else:
        file_path = str(main_directory + custom_name)

    # os.chmod(main_directory)
    dict_results = dictionary_to_save

    add_end = ''

    if saving_type == 'pickle':
        if (file_path[len(file_path) - 4:len(file_path)] != '.pkl'):
            add_end = '.pkl'
        with open(file_path + add_end, 'wb') as f:
            pickle.dump(dict_results, f, pickle.HIGHEST_PROTOCOL)


    elif saving_type == 'json':
        if (file_path[len(file_path) - 5:len(file_path)] != '.json'):
            add_end = '.json'

        # dictionary_to_save['json_encoder'] = _custom_JSONEncoder
        with open(file_path + add_end, 'w') as f:
            json.dump(dict_results, f, cls=_custom_JSONEncoder)




# Define functions for implementation

In [11]:
def _apply_elementary_gate_native(quantum_circuit:Circuit,
                                  qubit_index:int,
                                  gate_label: str):
    if gate_label.upper() == 'I':
        quantum_circuit+=Instruction(Gate.I(), qubit_index)
    elif gate_label.upper() == 'X':
        # X
        quantum_circuit = quantum_circuit.rz(qubit_index, np.pi)
        quantum_circuit = quantum_circuit.rx(qubit_index,np.pi / 2)
        quantum_circuit = quantum_circuit.rz(qubit_index, np.pi)
        quantum_circuit = quantum_circuit.rx(qubit_index,-np.pi / 2)
        
    elif gate_label.upper() == 'Y':
        # Y
        quantum_circuit = quantum_circuit.rz(qubit_index, -2.446121240088584)
        quantum_circuit = quantum_circuit.rx(qubit_index,np.pi / 2)
        quantum_circuit = quantum_circuit.rz(qubit_index, np.pi)
        quantum_circuit = quantum_circuit.rx(qubit_index,-np.pi / 2)
        quantum_circuit = quantum_circuit.rz(qubit_index, -2.446121240088584)
        
    elif gate_label.upper() == 'Z':
        # Z
        quantum_circuit = quantum_circuit.rz(qubit_index, np.pi)
        

    elif gate_label.upper() == 'H':
        # H
        quantum_circuit = quantum_circuit.rz(qubit_index, np.pi)
        quantum_circuit = quantum_circuit.rx(qubit_index,np.pi / 2)
        quantum_circuit = quantum_circuit.rz(qubit_index, np.pi/2)
        quantum_circuit = quantum_circuit.rx(qubit_index,-np.pi / 2)


    elif gate_label.upper() == 'S':
        # S
        quantum_circuit = quantum_circuit.rz(qubit_index, np.pi/2)


    elif gate_label.upper() == 'S*':
        # Sdag
        quantum_circuit = quantum_circuit.rz(qubit_index, -np.pi/2)

    elif gate_label.upper() == 'T':
        # T
        
        quantum_circuit = quantum_circuit.rz(qubit_index, np.pi/4)
    
    elif gate_label.upper() == 'T*':
        # Tdag
        quantum_circuit = quantum_circuit.rz(qubit_index, -np.pi/4)

    else:
        raise ValueError(f"Gate label: '{gate_label}' not recognized.")

    return quantum_circuit

def _apply_pauli_eigenstate(eigenstate_index:int,
                            quantum_circuit:Circuit,
                            qubit_index:int):
    
    #Z+
    if eigenstate_index == 0:
        quantum_circuit = _apply_elementary_gate_native(quantum_circuit=quantum_circuit,
                                                qubit_index=qubit_index,
                                                gate_label='I')
    
    #Z-
    elif eigenstate_index == 1:
        quantum_circuit = _apply_elementary_gate_native(quantum_circuit=quantum_circuit,
                                                qubit_index=qubit_index,
                                                gate_label='X')
        
    #X+
    elif eigenstate_index == 2:
        quantum_circuit = _apply_elementary_gate_native(quantum_circuit=quantum_circuit,
                                                qubit_index=qubit_index,
                                                gate_label='H')
    #Y+
    elif eigenstate_index == 3:
        quantum_circuit = _apply_elementary_gate_native(quantum_circuit=quantum_circuit,
                                                qubit_index=qubit_index,
                                                gate_label='H')
        quantum_circuit = _apply_elementary_gate_native(quantum_circuit=quantum_circuit,
                                                qubit_index=qubit_index,
                                                gate_label='S')

    #X-
    elif eigenstate_index == 4:
        quantum_circuit = _apply_elementary_gate_native(quantum_circuit=quantum_circuit,
                                                qubit_index=qubit_index,
                                                gate_label='X')
        quantum_circuit = _apply_elementary_gate_native(quantum_circuit=quantum_circuit,
                                                qubit_index=qubit_index,
                                                gate_label='H')
    #Y-
    elif eigenstate_index == 5:
        quantum_circuit = _apply_elementary_gate_native(quantum_circuit=quantum_circuit,
                                                qubit_index=qubit_index,
                                                gate_label='X')
        quantum_circuit = _apply_elementary_gate_native(quantum_circuit=quantum_circuit,
                                                qubit_index=qubit_index,
                                                gate_label='H')
        quantum_circuit = _apply_elementary_gate_native(quantum_circuit=quantum_circuit,
                                                qubit_index=qubit_index,
                                                gate_label='S')

    else:
        raise ValueError(f"Incorrect eigenstate index: '{eigenstate_index}'!")

    return quantum_circuit

## Check qubits on physical backend

In [12]:
device_true = AwsDevice("arn:aws:braket:::device/qpu/rigetti/Aspen-8")
calibration_data_1q = device_true.properties.provider.specs['1Q']


fidelity_threshold = 0.99


fidelities_dict = {}
high_fidelity_qubits = []
high_fidelity_qubits_tuples = []
for qubit_index in calibration_data_1q.keys():
    calibration_this_qubit = calibration_data_1q[qubit_index]

    fidelity_now = np.min([calibration_this_qubit['f1QRB'],
                          calibration_this_qubit['f1Q_simultaneous_RB']])
    
    fidelities_dict[qubit_index] = fidelity_now
    if fidelity_now>=fidelity_threshold:
        high_fidelity_qubits.append(qubit_index)
        high_fidelity_qubits_tuples.append((qubit_index, np.round(fidelity_now,3)))
        

cool_print("Number with high fidelity:",len(high_fidelity_qubits))
cool_print("List:")
print(high_fidelity_qubits_tuples)

# for stuff in device_true.properties.action['braket.ir.jaqcd.program'].supportedResultTypes:
#     print(stuff)

Number with high fidelity: 20
List:
[('1', 0.998), ('3', 0.995), ('4', 0.993), ('5', 0.996), ('7', 0.995), ('11', 0.992), ('13', 0.995), ('15', 0.991), ('16', 0.994), ('17', 0.993), ('22', 0.993), ('24', 0.995), ('25', 0.998), ('26', 0.992), ('30', 0.992), ('32', 0.996), ('33', 0.997), ('34', 0.996), ('35', 0.994), ('37', 0.994)]


## Specify details of experiments



The experiment is divided into 4 phases:
* We run 560 circuits that implement:
* * QDOT with k=2,
* * DDOT with k=4.
* We run 300 circuits that implement ground states of local Hamiltonians (2SAT with clause density 4).
* We run additional 200 characterization circuits that upgrade DDOT collection to k=5.
* If there is time left we may either:
* * implement more DDOT experiments,
* * implement more ground states of local hamiltonians.

Step 2 is between 1 and 3 to mitigate the risk that due to time constraints we have a lot of characterization results but very few ground states.


Please note that DDOT collections were created for 30 qubits, but by definition they also implement DDOT for lower numbers of qubits. So do not worry that lengths of strings describing experiments are higher than number of qubits.



In [13]:
#how long to wait for results
poll_timeout_seconds=5*24*60*60
#how often check if task completed
poll_interval_seconds=1


#get today's date
date_name = get_today_date()


### REMEMBER TO CHANGE TO PROPER QUBITS
qubits_to_implement = [int(x) for x in high_fidelity_qubits]
#qubits_to_implement = list(range(15))
qubits_enumerated = dict(enumerate(qubits_to_implement))
number_of_qubits = len(qubits_to_implement)


#REMEMBER TO SET THIS
number_of_shots = 10**4
#number_of_shots = 10**3


#So turns out that AWS has this option of running batches -
#perhaps this is faster. Try running with this and if something seems wrong, set this to False
run_with_parallelization = True


#additional experimental index for saving
experiment_index_additional = 0


#REMEMBER TO CHOOSE PROPER BACKEND
#backend_name = 'simulation'
backend_name = 'ASPEN-9'

#SEE BELOW FOR AVVAILABLE NAMES
experiments_name = 'CHARACTERIZATION-BATCH-0'


if backend_name.upper()=='SIMULATION':
    device = AwsDevice("arn:aws:braket:::device/quantum-simulator/amazon/sv1")
elif backend_name.upper()=='ASPEN-9':
    device = AwsDevice("arn:aws:braket:::device/qpu/rigetti/Aspen-9")



if experiments_name.upper() == 'TEST':
    labels_of_experiments = experimental_labels.___test_simulator___
    
elif experiments_name.upper()=='CHARACTERIZATION-BATCH-0':
    labels_of_experiments = experimental_labels.___characterization_batch0___
elif experiments_name.upper()=='CHARACTERIZATION-BATCH-1':
    labels_of_experiments = experimental_labels.___characterization_batch1___
elif experiments_name.upper()=='CHARACTERIZATION-BATCH-2':
    labels_of_experiments = experimental_labels.___characterization_batch2___
    
    
elif experiments_name.upper()=='HAMILTONIANS-2SAT-BATCH-0':
    directory_ground_states = get_module_directory()+"saved_data/stored_hamiltonians/2SAT/"
    with open(directory_ground_states + 'ground_states_2SAT.pkl', 'rb') as filein:
        ___ground_states_2SAT___ = pickle.load(filein)
        
    labels_of_experiments = list(___ground_states_2SAT___['N%s'%number_of_qubits].values())[0:300]
    
elif experiments_name.upper()=='HAMILTONIANS-2SAT-BATCH-1':
    directory_ground_states = get_module_directory()+"saved_data/stored_hamiltonians/2SAT/"
    with open(directory_ground_states + 'ground_states_2SAT.pkl', 'rb') as filein:
        ___ground_states_2SAT___ = pickle.load(filein)

    labels_of_experiments = list(___ground_states_2SAT___['N%s'%number_of_qubits].values())[300:-1]

                                 

                                 

#specify path to amazon bucket; DO NOT CHANGE THIS
bucket_folder = f"{backend_name}/{experiments_name}/{date_name}/"
bucket_name = 'amazon-braket-dfc1bac8c752' # the name of the bucket
s3_folder = (bucket_name, bucket_folder)

    
#specify where to save results
path_to_save_results_all = get_module_directory()+f"saved_data/{date_name}/{backend_name}/N{number_of_qubits}/{experiments_name}/{number_of_shots}/no-{experiment_index_additional}/"

path_to_save_results_raw = path_to_save_results_all+'raw/'
path_to_save_results_metadata = path_to_save_results_all+'metadata/'

global_experiment_prefix = experiments_name

In [14]:

#create dictionary with circuits to be implemented
circuits_dictionary = {}

#go through experimental labels
for exp_label in labels_of_experiments: 
    #get instance of circuit
    circuit_now_not_verbatim = Circuit()
    
    #go through qubit indices
    for qubit_counter, qubit_true_index in qubits_enumerated.items():
        #check what Pauli eigenstate to implement
        eigenstate_index_now = exp_label[qubit_counter]
        
        #apply gate that applies this pauli eigenstate
        circuit_now_not_verbatim = _apply_pauli_eigenstate(eigenstate_index=int(eigenstate_index_now),
                                    quantum_circuit=circuit_now_not_verbatim,
                                    qubit_index=qubit_true_index)
        
    #add verbatim compilation command
    #simulator does not support this...
    if backend_name.upper()=='SIMULATION':
        circuit_now_verbatim = circuit_now_not_verbatim
    else:
        circuit_now_verbatim = Circuit().add_verbatim_box(circuit_now_not_verbatim)
    
    
    #CHANGE HERE IF VERBATIM DOES NOT WORK
    circuit_now_to_append = circuit_now_verbatim
    
    
    #CHANGE HERE IF YOU DON't WANT TO PRINT CIRCUITS
    cool_print("Experiment label:",exp_label)
    print(circuit_now_to_append)


    circuits_dictionary[exp_label] = circuit_now_to_append

Experiment label: '340340504202133251155213513211'
T   : |      0      |   1    |   2    |   3    |    4    |   5    |   6    |   7    |    8    |   9    |    10     |
                                                                                                                    
q1  : -StartVerbatim-Rz(3.14)-Rx(1.57)-Rz(1.57)-Rx(-1.57)-Rz(1.57)--------------------------------------EndVerbatim-
       |                                                                                                |           
q3  : -|-------------Rz(3.14)-Rx(1.57)-Rz(3.14)-Rx(-1.57)-Rz(3.14)-Rx(1.57)-Rz(1.57)-Rx(-1.57)----------|-----------
       |                                                                                                |           
q4  : -|-------------I----------------------------------------------------------------------------------|-----------
       |                                                                                                |           
q5  : -|-----

Experiment label: '133013141340511100500120105555'
T   : |      0      |   1    |   2    |   3    |    4    |   5    |   6    |   7    |    8    |   9    |    10     |
                                                                                                                    
q1  : -StartVerbatim-Rz(3.14)-Rx(1.57)-Rz(3.14)-Rx(-1.57)-----------------------------------------------EndVerbatim-
       |                                                                                                |           
q3  : -|-------------Rz(3.14)-Rx(1.57)-Rz(1.57)-Rx(-1.57)-Rz(1.57)--------------------------------------|-----------
       |                                                                                                |           
q4  : -|-------------Rz(3.14)-Rx(1.57)-Rz(1.57)-Rx(-1.57)-Rz(1.57)--------------------------------------|-----------
       |                                                                                                |           
q5  : -|-----

T   : |      0      |   1    |   2    |   3    |    4    |   5    |   6    |   7    |    8    |   9    |    10     |
                                                                                                                    
q1  : -StartVerbatim-Rz(3.14)-Rx(1.57)-Rz(3.14)-Rx(-1.57)-----------------------------------------------EndVerbatim-
       |                                                                                                |           
q3  : -|-------------Rz(3.14)-Rx(1.57)-Rz(3.14)-Rx(-1.57)-----------------------------------------------|-----------
       |                                                                                                |           
q4  : -|-------------Rz(3.14)-Rx(1.57)-Rz(1.57)-Rx(-1.57)-Rz(1.57)--------------------------------------|-----------
       |                                                                                                |           
q5  : -|-------------I------------------------------------------

T   : |      0      |   1    |   2    |   3    |    4    |   5    |   6    |   7    |    8    |   9    |    10     |
                                                                                                                    
q1  : -StartVerbatim-I----------------------------------------------------------------------------------EndVerbatim-
       |                                                                                                |           
q3  : -|-------------I----------------------------------------------------------------------------------|-----------
       |                                                                                                |           
q4  : -|-------------Rz(3.14)-Rx(1.57)-Rz(1.57)-Rx(-1.57)-----------------------------------------------|-----------
       |                                                                                                |           
q5  : -|-------------Rz(3.14)-Rx(1.57)-Rz(3.14)-Rx(-1.57)-Rz(3.1

T   : |      0      |   1    |   2    |   3    |    4    |   5    |   6    |   7    |    8    |   9    |    10     |
Experiment label: '451444240432405045000011242415'
T   : |      0      |   1    |   2    |   3    |    4    |   5    |   6    |   7    |    8    |   9    |    10     |
                                                                                                                    
q1  : -StartVerbatim-Rz(3.14)-Rx(1.57)-Rz(3.14)-Rx(-1.57)-Rz(3.14)-Rx(1.57)-Rz(1.57)-Rx(-1.57)----------EndVerbatim-
       |                                                                                                |           
q3  : -|-------------Rz(3.14)-Rx(1.57)-Rz(3.14)-Rx(-1.57)-Rz(3.14)-Rx(1.57)-Rz(1.57)-Rx(-1.57)-Rz(1.57)-|-----------
       |                                                                                                |           
q4  : -|-------------Rz(3.14)-Rx(1.57)-Rz(3.14)-Rx(-1.57)-----------------------------------------------|-----------
       |     

...

Experiment label: '101001111111000101000100010010'
T   : |      0      |   1    |   2    |   3    |    4    |     5     |
                                                                      
q1  : -StartVerbatim-Rz(3.14)-Rx(1.57)-Rz(3.14)-Rx(-1.57)-EndVerbatim-
       |                                                  |           
q3  : -|-------------I------------------------------------|-----------
       |                                                  |           
q4  : -|-------------Rz(3.14)-Rx(1.57)-Rz(3.14)-Rx(-1.57)-|-----------
       |                                                  |           
q5  : -|-------------I------------------------------------|-----------
       |                                                  |           
q7  : -|-------------I------------------------------------|-----------
       |                                                  |           
q11 : -|-------------Rz(3.14)-Rx(1.57)-Rz(3.14)-Rx(-1.57)-|-----------
       |                  

Experiment label: '001010001011100010110100000000'
T   : |      0      |   1    |   2    |   3    |    4    |     5     |
                                                                      
q1  : -StartVerbatim-I------------------------------------EndVerbatim-
       |                                                  |           
q3  : -|-------------I------------------------------------|-----------
       |                                                  |           
q4  : -|-------------Rz(3.14)-Rx(1.57)-Rz(3.14)-Rx(-1.57)-|-----------
       |                                                  |           
q5  : -|-------------I------------------------------------|-----------
       |                                                  |           
q7  : -|-------------Rz(3.14)-Rx(1.57)-Rz(3.14)-Rx(-1.57)-|-----------
       |                                                  |           
q11 : -|-------------I------------------------------------|-----------
       |                  

Experiment label: '000010001001010010101100100110'
T   : |      0      |   1    |   2    |   3    |    4    |     5     |
                                                                      
q1  : -StartVerbatim-I------------------------------------EndVerbatim-
       |                                                  |           
q3  : -|-------------I------------------------------------|-----------
       |                                                  |           
q4  : -|-------------I------------------------------------|-----------
       |                                                  |           
q5  : -|-------------I------------------------------------|-----------
       |                                                  |           
q7  : -|-------------Rz(3.14)-Rx(1.57)-Rz(3.14)-Rx(-1.57)-|-----------
       |                                                  |           
q11 : -|-------------I------------------------------------|-----------
       |                  

Experiment label: '001101110010001001100001000010'
T   : |      0      |   1    |   2    |   3    |    4    |     5     |
                                                                      
q1  : -StartVerbatim-I------------------------------------EndVerbatim-
       |                                                  |           
q3  : -|-------------I------------------------------------|-----------
       |                                                  |           
q4  : -|-------------Rz(3.14)-Rx(1.57)-Rz(3.14)-Rx(-1.57)-|-----------
       |                                                  |           
q5  : -|-------------Rz(3.14)-Rx(1.57)-Rz(3.14)-Rx(-1.57)-|-----------
       |                                                  |           
q7  : -|-------------I------------------------------------|-----------
       |                                                  |           
q11 : -|-------------Rz(3.14)-Rx(1.57)-Rz(3.14)-Rx(-1.57)-|-----------
       |                  

Experiment label: '010101100110011011111111000000'
T   : |      0      |   1    |   2    |   3    |    4    |     5     |
                                                                      
q1  : -StartVerbatim-I------------------------------------EndVerbatim-
       |                                                  |           
q3  : -|-------------Rz(3.14)-Rx(1.57)-Rz(3.14)-Rx(-1.57)-|-----------
       |                                                  |           
q4  : -|-------------I------------------------------------|-----------
       |                                                  |           
q5  : -|-------------Rz(3.14)-Rx(1.57)-Rz(3.14)-Rx(-1.57)-|-----------
       |                                                  |           
q7  : -|-------------I------------------------------------|-----------
       |                                                  |           
q11 : -|-------------Rz(3.14)-Rx(1.57)-Rz(3.14)-Rx(-1.57)-|-----------
       |                  

## CHECK IF SOME EXPERIMENTS WERE ALREADY IMPLEMENTED

In [15]:
already_done = []

#check if directory with results exists at all
try:
    stuff_inside = sorted(os.listdir(path_to_save_results_raw))
except(FileNotFoundError):
    stuff_inside = []
    
    
#check what already exists in directory with results
for file_saved in stuff_inside:
    time_stamp = get_time_now()
    
    name_to_save_results =  f"{global_experiment_prefix}_{exp_label}_date-{time_stamp}"
          
    name_of_experiment = file_saved[len(global_experiment_prefix)+1:-(6+len(time_stamp)+4)]
    
    already_done.append(name_of_experiment)

#be sure to not duplicate experiments that were performed multiple times
already_done = list(np.unique(already_done))

#get list with labels that were not implemented
labels_of_experiments_not_done = [exp_label for exp_label in labels_of_experiments if exp_label not in already_done]


#divide experiments into batches for parallelized implementation
if run_with_parallelization:
    batch_size = 10
else:
    batch_size = 1
    
labels_of_experiments_not_done_batches = [labels_of_experiments_not_done[i:i + batch_size] for i in range(0, len(labels_of_experiments_not_done), batch_size)]


## RUN EXPERIMENTS

In [ ]:
cool_print("Already done:", f"{len(already_done)}/{len(labels_of_experiments)}")
cool_print("Planned now:", f"{len(labels_of_experiments_not_done)}/{len(labels_of_experiments)}")
cool_print("Qubits to implement:",qubits_to_implement)
cool_print("Backend:",backend_name)
cool_print('Number of shots:',number_of_shots)
cool_print("Name of experiments:",experiments_name)
cool_print("Additional experimental index:",experiment_index_additional)
cool_print("Parallelized implementation:",run_with_parallelization)


t000 = time.time()
while already_done != labels_of_experiments:     
    for experiments_batch_labels in labels_of_experiments_not_done_batches:
        experiments_batch = [circuits_dictionary[exp_label] for exp_label in experiments_batch_labels]
        t0_now = time.time()
        
        cool_print(f"___________\nGOT LABELS: '{experiments_batch_labels}':",'RUNNING!')
    
        if run_with_parallelization:
            #try to implement experiment and if there is error, wait 10 s then try again
            for error_counter in range(1000):
                try:
                    if backend_name.upper()=='SIMULATION':
                        task_now = device.run_batch(task_specifications=experiments_batch,
                                              s3_destination_folder=s3_folder,
                                              shots=number_of_shots,
                                              poll_timeout_seconds=poll_timeout_seconds,
                                              poll_interval_seconds=poll_interval_seconds,
                                             disable_qubit_rewiring=True)

                    elif backend_name.upper()=='ASPEN-9':
                        task_now = device.run_batch(task_specifications=experiments_batch, 
                                              s3_destination_folder=s3_folder,
                                              shots=number_of_shots,
                                              poll_timeout_seconds=poll_timeout_seconds,
                                             poll_interval_seconds=poll_interval_seconds,
                                             disable_qubit_rewiring=True)
                    
                    tasks_list = task_now._tasks

                    # get results from task
                    results_list = task_now.results(fail_unsuccessful=True)
                    

                    break
                except:
                    print(
                        f"WARNING: An error occurred for experiment: '{experiments_batch_labels}'.")
                    traceback.print_exc()

                print(f"Waiting {10}s.")
                time.sleep(10)
                print(f"Retrying experiment: '{experiments_batch_labels}'.")

                if (error_counter + 1) % 25 == 0:
                    cool_print(
                        f"\n\nAlready tried '{experiments_batch_labels}' {str(error_counter)} times! Something seems to be wrong...\n\n",
                        '',
                        'red')
                    
                    
        else:
            tasks_list = []
            results_list = []
            for exp_label in experiments_batch_labels:
                #try to implement experiment and if there is error, wait 10 s then try again
                for error_counter in range(1000):
                    try:
                        if backend_name.upper()=='SIMULATION':
                            task_now = device.run(task_specification=circuits_dictionary[exp_label],
                                                  s3_destination_folder=s3_folder,
                                                  shots=number_of_shots,
                                                  poll_timeout_seconds=poll_timeout_seconds,
                                                  poll_interval_seconds=poll_interval_seconds,
                                                 disable_qubit_rewiring=True)

                        elif backend_name.upper()=='ASPEN-9':
                            task_now = device.run(task_specification=circuits_dictionary[exp_label], 
                                                  s3_destination_folder=s3_folder,
                                                  shots=number_of_shots,
                                                  poll_timeout_seconds=poll_timeout_seconds,
                                                 poll_interval_seconds=poll_interval_seconds,
                                                 disable_qubit_rewiring=True)

                        tasks_list.append(task_now)
                        results_list.append(task_now.result())


                        break
                    except:
                        print(
                            f"WARNING: An error occurred for experiment: '{exp_label}'.")
                        traceback.print_exc()

                    print(f"Waiting {10}s.")
                    time.sleep(10)
                    print(f"Retrying experiment: '{exp_label}'.")

                    if (error_counter + 1) % 25 == 0:
                        cool_print(
                            f"\n\nAlready tried '{exp_label}' {str(error_counter)} times! Something seems to be wrong...\n\n",
                            '',
                            'red')

        time_took_now = time.time()-t0_now  
        
        
        target_ids = [task.id for task in tasks_list]

        for exp_label_index in range(len(experiments_batch_labels)):  
            exp_label = experiments_batch_labels[exp_label_index]
            task_now = tasks_list[exp_label_index]
            results_now = results_list[exp_label_index]
            
            task_id = target_ids[exp_label_index]            
            


            circuit_now = circuits_dictionary[exp_label]


            #get metadata of implemented job
            id_implemented = results_now.task_metadata.id
            measured_qubits = results_now.measured_qubits
            rewiring_disabled = results_now.task_metadata.deviceParameters.paradigmParameters.disableQubitRewiring


            #check if metadata is consitent with what we wanted to do
            if not id_implemented==task_id:
                cool_print("Something wrong with id:",'','red')
                print("target:",task_id)
                print("implemented:",id_implemented)

            if not rewiring_disabled:
                cool_print("Qubits rewiring has not been disabled!",'','red')

            if measured_qubits!=qubits_to_implement:
                cool_print("Something wrong with measured qubits!",'','red')
                cool_print("Target qubits:",qubits_to_implement,'red')
                cool_print("Implemented qubits:",measured_qubits,'red')


            #get results in form of bitstrings and counts dictionary
            bits_array_now = results_now.measurements
            counts_now = dict(results_now.measurement_counts)


                  
            time_stamp = get_time_now()

            #specify dictionary to be saved (without circuit) - format compatible with future pickling
            dictionary_to_save_raw = {'task_id_target':task_id,
                                 'task_id_implemented':id_implemented,
                                 'target_qubits':qubits_to_implement,
                                 'measured_qubits':measured_qubits,
                                 'rewiring_disabled':rewiring_disabled,
                                 'counts_dictionary':counts_now,
                                 'bits_array':bits_array_now,
                                 'time_took':time_took_now,
                                 'time_stamp':time_stamp,
                                 'experiment_label':exp_label}
            #specify dictionary to be saved (with circuit) - format potentially incompatible with future pickling
            dictionary_to_save_metadata = {**dictionary_to_save_raw,
                                           **{'circuit':circuit_now}}

            #name of file with results (adding time stamp prohibits accidental overwrites)
            name_to_save_results =  f"{global_experiment_prefix}_{exp_label}_date-{time_stamp}"


            #save raw results
            save_results(dictionary_to_save=dictionary_to_save_raw,
                     directory=path_to_save_results_raw,
                     saving_type='pickle',
                     custom_name = name_to_save_results,
                     get_cwd=False)

            #save results with circuit
            save_results(dictionary_to_save=dictionary_to_save_metadata,
             directory=path_to_save_results_metadata,
             saving_type='pickle',
             custom_name = name_to_save_results,
             get_cwd=False)


            already_done.append(exp_label)
            
        

        #print time
        if np.floor(time_took_now)<60:
            time_to_print = f"{np.round(time_took_now,3)} s"
        else:
            time_to_print = f"{np.round(time_took_now/60,3)} m"

        cool_print(f"SUCESFULLY IMPLEMENTED EXPERIMENTS: '{experiments_batch_labels}' in time {time_to_print}.",
                      '','green')
        cool_print(f"Time spent so far: {np.round(time.time()-t000,3)} s.",
              '','green')
        print()


        
cool_print("\nALL FINISHED!",'','red')
cool_print("Everything took:",np.round((time.time()-t000)/60,3))

Already done: '0/560'
Planned now: '560/560'
Qubits to implement: [1, 3, 4, 5, 7, 11, 13, 15, 16, 17, 22, 24, 25, 26, 30, 32, 33, 34, 35, 37]
Backend: 'ASPEN-9'
Number of shots: 10000
Name of experiments: 'CHARACTERIZATION-BATCH-0'
Additional experimental index: 0
Parallelized implementation: True
Do you wish to run experiments? [y/n]
y
___________
GOT LABELS: '['340340504202133251155213513211', '104124441302230530500305111000', '133013141340511100500120105555', '210141153232543411411414112240', '543514505021045234115305134453', '343400225030230043542354355052', '115113445004244145102341045320', '520441225245312540212240353543', '354042354023154100244455500231', '155102254251511543050321400405']': 'RUNNING!'
Attention: Directory "/home/ec2-user/SageMaker/saved_data/2021-09-28/ASPEN-9/N20/CHARACTERIZATION-BATCH-0/10000/no-0/raw/" was created.
Attention: Directory "/home/ec2-user/SageMaker/saved_data/2021-09-28/ASPEN-9/N20/CHARACTERIZATION-BATCH-0/10000/no-0/metadata/" was created.
SUCES